In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
standings_urls = ["https://fbref.com/en/comps/9/Premier-League-Stats",
    "https://fbref.com/en/comps/11/Serie-A-Stats",
    "https://fbref.com/en/comps/12/La-Liga-Stats",
    "https://fbref.com/en/comps/13/Ligue-1-Stats",
    "https://fbref.com/en/comps/20/Bundesliga-Stats"]

leagues =  pd.read_excel('./../Club_name.xlsx')
match_urls = []
temp = []

In [3]:
leagues.head()

,Premier League,Serie A,La Liga,Ligue 1,Bundesliga
0,Arsenal,Napoli,Barcelona,Paris S-G,Bayern Munich
1,Manchester City,Inter,Real Madrid,Marseille,Union Berlin
2,Manchester Utd,Roma,Real Sociedad,Lens,Dortmund
3,Newcastle Utd,Lazio,AtlÃ©tico Madrid,Monaco,RB Leipzig
4,Tottenham,Atalanta,Rayo Vallecano,Rennes,Eint Frankfurt


In [4]:
count = 0

In [5]:
def get_match_url(rows, league):
    rows = [row for row in rows if row.find_all('td')[4].get_text() =='Home']
    rows = [row for row in rows if row.find_all('td')[1].get_text() == league] 
    links = [row.find_all('a')[-1] for row in rows]
    links = [l.get('href') for l in links if l.get_text() == 'Match Report']
    match_urls = [f"https://fbref.com{l}" for l in links]
    return match_urls

In [16]:
match_urls = []
for standings_url, league in zip(standings_urls,leagues):
    data = requests.get(standings_url) 
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]
    links = standings_table.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if '/squads/' in l]
    standing = pd.read_html(data.text, attrs={'class': 'stats_table'})[0]

    team_urls = [f"https://fbref.com{l}" for l in links]
    for i, (l, team_name) in enumerate((zip(team_urls, leagues[league]))):
        if i == len(team_urls): 
            break     
        d = requests.get(l)
        soup_club = BeautifulSoup(d.text)
        match_logs = soup_club.select('table.stats_table')[1]
        all_rows = match_logs.select('tr')[1:]
        match_urls.extend(get_match_url(all_rows, league))
        count += 1
    temp = match_urls
    print(len(match_urls))

209
419
619
839
1010


In [25]:
print(len(match_urls))

1010


In [24]:
file = open("url.txt", "w")
for url in match_urls:
    file.write(url+'\n')

In [ ]:
for url in match_urls:
    data = requests.get(url)
    soup = BeautifulSoup(data.text)
    home, away = soup.select('table.stats_table')[1:6], soup.select('table.stats_table')[8:12]
    